# SWMAL Exercise


## Hyperparameters and Gridsearch 

Machine learning models have certain global parameters which decide on the inner workings of the model. An example of this could be a degree of polynomial models, or number of neurons or hidden layers in neural network models. Choosing the optimal hyperparameters for machine learning models manually is extremely time consuming, since it would involve a silly amount of trial and error. In this exercise we will delve into optimizing the hyperparameters using GridSearch and RandomizedSearch.

### Qa Explain GridSearchCV

The following python code block sets up our functions to load and set up the data, as well as display results of a gridsearch. See detailed explanation in the comments.


In [1]:
# Explanation:
# This block of code loads the data and defines functions which will be used to present results of gridsearch
# GetBestModelCTOR() returns a string with a constructor of the model with the best parameters in it
# SearchReport() displays the best models name, its best parameters, score and index. It also asserts that the scoring system used is f1_micro.
# ClassificationReport() uses the model to predict with the test data supplied in parameters. It then compares the prediction with true values.
# TryKerasImport() asserts that keras module is loaded and ready to be used
# LoadAndSetupData() loads the data and reshapes it if needed, chosen by the parameter 'mode' - either iris, mnist or moon dataset

from time import time
import numpy as np
import sys

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

# Need this to import libitmal on MY Windows machine. Replace with your GITMAL directory or uncomment if you already have gitmal in pythonpath  - Marcin
sys.path.append("C:\\UNI_2023\\ml\\gitmal")

from libitmal import dataloaders as itmaldataloaders # Needed for load of iris, moon and mnist

currmode="N/A" # GLOBAL var!

def SearchReport(model):

    # This method
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            ret_str=""
            for key in sorted(best_params):
                value = best_params[key]
                temp_str = "'" if str(type(value))=="<class 'str'>" else ""
                if len(ret_str)>0:
                    ret_str += ','
                ret_str += f'{key}={temp_str}{value}{temp_str}'
            return ret_str
        try:
            param_str = GetParams(best_params)
            return type(model).__name__ + '(' + param_str + ')'
        except:
            return "N/A(1)"

    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")

    global currmode
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred, target_names=target_names))
    print()

def FullReport(model, X_test, y_test, t):
    print(f"SEARCH TIME: {t:0.2f} sec")
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel

def LoadAndSetupData(mode, test_size=0.3):
    assert test_size>=0.0 and test_size<=1.0

    def ShapeToString(Z):
        n = Z.ndim
        s = "("
        for i in range(n):
            s += f"{Z.shape[i]:5d}"
            if i+1!=n:
                s += ";"
        return s+")"

    global currmode
    currmode=mode
    print(f"DATA: {currmode}..")

    if mode=='moon':
        X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
        itmaldataloaders.MOON_Plot(X, y)
    elif mode=='mnist':
        X, y = itmaldataloaders.MNIST_GetDataSet(load_mode=0)
        if X.ndim==3:
            X=np.reshape(X, (X.shape[0], -1))
    elif mode=='iris':
        X, y = itmaldataloaders.IRIS_GetDataSet()
    else:
        raise ValueError(f"could not load data for that particular mode='{mode}', only 'moon'/'mnist'/'iris' supported")

    print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

    assert X.ndim==2
    assert X.shape[0]==y.shape[0]
    assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=0, shuffle=True
    )

    print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
    print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
    print()

    return X_train, X_test, y_train, y_test

def TryKerasImport(verbose=True):

    kerasok = True
    try:
        import keras as keras_try
    except:
        kerasok = False

    tensorflowkerasok = True
    try:
        import tensorflow.keras as tensorflowkeras_try
    except:
        tensorflowkerasok = False

    ok = kerasok or tensorflowkerasok

    if not ok and verbose:
        if not kerasok:
            print("WARNING: importing 'keras' failed", file=sys.stderr)
        if not tensorflowkerasok:
            print("WARNING: importing 'tensorflow.keras' failed", file=sys.stderr)

    return ok

print(f"OK(function setup" + ("" if TryKerasImport() else ", hope MNIST loads works because it seems you miss the installation of Keras or Tensorflow!") + ")")

OK(function setup, hope MNIST loads works because it seems you miss the installation of Keras or Tensorflow!)


Gridsearch is a method for tuning the hyperparameters of a model automatically, and GridSearchCV is the scikit-learn class that provides this functionality. To use GridSearchCV you simply supply it with the parameters you want it to test, and with values that you want to check. After running GridSearchCV.fit() on a dataset, gridsearch will go through all the possible combination of hyperparameters with the values supplied to it and compare their scoring using a scoring method of your choice. When that is done, the best parameters and the scores will be available in the GridSearchCV object. 

The following code block performs the actual grid search and displays the results using the functions supplied in the previous block. See the code comments for detailed explanation.

In [5]:
# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData('iris')  # 'iris', 'moon', or 'mnist'

# This is the model type we will test
model = svm.SVC(
    gamma=0.001
)

# These are the parameters that we want gridsearch to evaluate
# They are setup as a Dict[name, vals] with name always being a string and vals being whatever type the parameter values are
# In this particular example we will compare kernels 'linear' and 'rbf' against each other with 'C' (the regularization parameter) values being 0.1, 1 and 10
# This means the model will be fit 2*3 times
tuning_parameters = {
    'kernel': ('linear', 'rbf'),
    'C': [0.1, 1, 10]
}

# This is the number of KFolds that gridsearch's cross-validation strategy will use
CV = 5
# Don't display any debug informations
VERBOSE = 0

# Create gridsearch model with hyperparameters tested specified above
# n_jobs is number of jobs ran in parallel when fitting the model: -1 uses all available processors according to sklearn's documentation
# job is a somewhat ambiguous term so what exactly this means depends on the backend implementation in sklearn

# 'f1_micro' scoring method is defined as the micro-averaged harmonic mean of precision and recall.
# According to https://www.visobyte.com/2023/05/precision-recall-and-f1-score-in-object-detection-how-are-they-calculated.html#:~:text=The%20F1%20Score%20is%20a%20harmonic%20mean%20of,%2A%20%28Precision%20%2A%20Recall%29%20%2F%20%28Precision%20%2B%20Recall%29
# The precision score measures the rate of false positives and the recall score measures how accurately it predicts/detects
grid_tuned = GridSearchCV(model,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)

# Find the best parameters and measure the time to do so using X_train, y_train from the iris dataset
start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

# Report result. Uses previously defined methods to print data about the model. Also runs the best model to predict (X_test, y_test) validating it.
b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

DATA: iris..
  org. data:  X.shape      =(  150;    4), y.shape      =(  150)
  train data: X_train.shape=(  105;    4), y_train.shape=(  105)
  test data:  X_test.shape =(   45;    4), y_test.shape =(   45)

SEARCH TIME: 1.98 sec

Best model set found on train set:

	best parameters={'C': 1, 'kernel': 'linear'}
	best 'f1_micro' score=0.9714285714285715
	best index=2

Best estimator CTOR:
	SVC(C=1, gamma=0.001, kernel='linear')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.962 (+/-0.093) for {'C': 0.1, 'kernel': 'linear'}
	[ 1]: 0.371 (+/-0.038) for {'C': 0.1, 'kernel': 'rbf'}
	[ 2]: 0.971 (+/-0.047) for {'C': 1, 'kernel': 'linear'}
	[ 3]: 0.695 (+/-0.047) for {'C': 1, 'kernel': 'rbf'}
	[ 4]: 0.952 (+/-0.085) for {'C': 10, 'kernel': 'linear'}
	[ 5]: 0.924 (+/-0.097) for {'C': 10, 'kernel': 'rbf'}

Detailed classification report:
	The model is trained on the full development set.
	The scores are computed on the full evaluation set.

              precision    recall  f1-score  

The best parameters for our model are C=1 with linear kernel, having a score of 0.97143

### Qb Hyperparameter Grid Search using an SDG classifier

We will now use grid search to tune parameters of a Stochastic Gradient Descent classifier. 

In [6]:
from sklearn import linear_model

sgd = linear_model.SGDClassifier()

# Notice different parameters, check sklearn documentation for SGDClassifier for description of these parameters
tuning_parameters = {
    'loss': ['hinge', 'log_loss', 'perceptron', 'modified_huber', 'squared_error', 'huber'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 100],
    'tol' : [0.0001, 0.001, 0.01, 0.1],
    'max_iter' : [int(1e+5), int(1e+6)]
}

# This part is almost the same as in previous code block, we just swap the model
grid_tuned = GridSearchCV(sgd,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)

start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

SEARCH TIME: 120.97 sec

Best model set found on train set:

	best parameters={'alpha': 0.001, 'loss': 'modified_huber', 'max_iter': 100000, 'penalty': 'l1', 'tol': 0.01}
	best 'f1_micro' score=0.9904761904761905
	best index=222

Best estimator CTOR:
	SGDClassifier(alpha=0.001, loss='modified_huber', max_iter=100000, penalty='l1',
              tol=0.01)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.790 (+/-0.411) for {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 100000, 'penalty': 'l2', 'tol': 0.0001}
	[ 1]: 0.686 (+/-0.273) for {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 100000, 'penalty': 'l2', 'tol': 0.001}
	[ 2]: 0.800 (+/-0.194) for {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 100000, 'penalty': 'l2', 'tol': 0.01}
	[ 3]: 0.771 (+/-0.220) for {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 100000, 'penalty': 'l2', 'tol': 0.1}
	[ 4]: 0.819 (+/-0.279) for {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 100000, 'penalty': 'l1', 'tol': 0.0001}
	[ 5]: 0.952 (+/-0.085) for 

The best model has a staggering score of 0.99048, even better than the SVC model we used previously! It did take a few minutes though...

### Qc Hyperparameter Random  Search using an SDG classifier

Another method of finding the optimal parameters is to uze RandomizedSearchCV. In contrast to GridSearchCV it only chooses a few randomly selected samples of each hyperparameter provided, controlled by its own parameter 'n_iter'. This makes it potentially faster then GridSearchCV, but the results might not give the best scores. 

Below is an implementation and a test run of a RandomizedSearchCV.

In [7]:
random_tuned = RandomizedSearchCV(
    sgd,
    tuning_parameters,
    # Pick up to 20 different samples
    n_iter=20,
    # same state should give same rng distribution
    random_state=42,
    cv=CV,
    scoring='f1_micro',
    verbose=VERBOSE,
    n_jobs=-1
)

start = time()
random_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(random_tuned, X_test, y_test, t)
print('OK(random-search)')


SEARCH TIME: 0.12 sec

Best model set found on train set:

	best parameters={'tol': 0.01, 'penalty': 'l1', 'max_iter': 100000, 'loss': 'perceptron', 'alpha': 0.001}
	best 'f1_micro' score=0.9714285714285715
	best index=0

Best estimator CTOR:
	SGDClassifier(alpha=0.001, loss='perceptron', max_iter=100000, penalty='l1',
              tol=0.01)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.971 (+/-0.114) for {'tol': 0.01, 'penalty': 'l1', 'max_iter': 100000, 'loss': 'perceptron', 'alpha': 0.001}
	[ 1]: 0.352 (+/-0.076) for {'tol': 0.0001, 'penalty': 'l2', 'max_iter': 100000, 'loss': 'hinge', 'alpha': 100}
	[ 2]: 0.743 (+/-0.129) for {'tol': 0.1, 'penalty': 'elasticnet', 'max_iter': 100000, 'loss': 'perceptron', 'alpha': 1}
	[ 3]: 0.886 (+/-0.196) for {'tol': 0.001, 'penalty': 'elasticnet', 'max_iter': 1000000, 'loss': 'hinge', 'alpha': 0.01}
	[ 4]: 0.686 (+/-0.322) for {'tol': 0.1, 'penalty': 'elasticnet', 'max_iter': 1000000, 'loss': 'hinge', 'alpha': 0.0001}
	[ 5]: 0.895 (+/-0

We can see in the output that only 20 different combinations of parameters were tested, and the best parameters were not the same as in GridSearchCV test. The f1_score is also lower at 0.914 instead of 0.990. This could potentially be improved by adding more iterations.

## Qd MNIST Search Quest II


It's time to embark on an epic adventure of finding the best model for the MNIST dataset. Here we will use gridsearch on several models to attempt to find the best model according to the f1_micro scoring metric. The best scores across different groups will then be compared to each other. May the best man win!

In our first try, we'd like to test how fast the gridsearch performs, so we're gonna use a GridSearchCV with our existing SGDClassifier. Since complete GridSearch would take forever, we will only select a few parameters from each section.

In [2]:
# Load mnist data separate block, since this can take extra time
X_train, X_test, y_train, y_test = LoadAndSetupData('mnist')


DATA: mnist..


C:\UNI_2023\ml\gitmal\libitmal\dataloaders.py:65: UserWarning: MNIST_GetDataSet(): failed to import and load data in load_mode 'tensorflow.keras', proceding to next mode..
  warnings.warn("MNIST_GetDataSet(): failed to import and load data in load_mode 'tensorflow.keras', proceding to next mode..")
C:\UNI_2023\ml\gitmal\libitmal\dataloaders.py:77: UserWarning: MNIST_GetDataSet(): failed to import and load data in load_mode 'keras', proceding to next mode..
  warnings.warn("MNIST_GetDataSet(): failed to import and load data in load_mode 'keras', proceding to next mode..")
c:\Users\nastr\anaconda3\envs\swmal\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Secti

  org. data:  X.shape      =(70000;  784), y.shape      =(70000)
  train data: X_train.shape=(49000;  784), y_train.shape=(49000)
  test data:  X_test.shape =(21000;  784), y_test.shape =(21000)



In [9]:
# Notice different parameters, check sklearn documentation for SGDClassifier for description of these parameters
tuning_parameters = {
    'loss': ['hinge', 'log_loss'],
    'penalty': ['l2', 'l1'],
    'alpha' : [0.001, 0.01, 0.1],
    'tol' : [0.001, 0.01, 0.1],
    'max_iter' : [int(1e+2), int(1e+3)]
}

# This part is almost the same as in previous code block, we just swap the model
grid_tuned = GridSearchCV(sgd,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)

start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)

SEARCH TIME: 17656.98 sec

Best model set found on train set:

	best parameters={'alpha': 0.001, 'loss': 'log_loss', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.1}
	best 'f1_micro' score=0.8937959183673468
	best index=17

Best estimator CTOR:
	SGDClassifier(alpha=0.001, loss='log_loss', max_iter=100, penalty='l1', tol=0.1)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.868 (+/-0.017) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.001}
	[ 1]: 0.864 (+/-0.027) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.01}
	[ 2]: 0.882 (+/-0.012) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.1}
	[ 3]: 0.893 (+/-0.008) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.001}
	[ 4]: 0.892 (+/-0.007) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.01}
	[ 5]: 0.891 (+/-0.003) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'pena

c:\Users\nastr\anaconda3\envs\swmal\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


A score of 0.8938 isn't terrible but it surely can be improved upon. Let's try a model which hasn't been explored during the lectures: RandomForestClassifier.

In [12]:
# Random forest gridsearch!
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()

tuning_parameters = {
    'n_estimators' : [50, 100, 1000],
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'max_depth' : [None, 2, 5, 10]
}

grid_tuned = GridSearchCV(forest,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)


start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)

c:\Users\nastr\anaconda3\envs\swmal\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 36 is smaller than n_iter=50. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 36 candidates, totalling 180 fits
SEARCH TIME: 5070.60 sec

Best model set found on train set:

	best parameters={'alpha': 0.001, 'loss': 'log_loss', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.1}
	best 'f1_micro' score=0.8937959183673468
	best index=17

Best estimator CTOR:
	SGDClassifier(alpha=0.001, loss='log_loss', max_iter=100, penalty='l1', tol=0.1)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.868 (+/-0.017) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.001}
	[ 1]: 0.864 (+/-0.027) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.01}
	[ 2]: 0.882 (+/-0.012) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l2', 'tol': 0.1}
	[ 3]: 0.893 (+/-0.008) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.001}
	[ 4]: 0.892 (+/-0.007) for {'alpha': 0.001, 'loss': 'hinge', 'max_iter': 100, 'penalty': 'l1', 'tol': 0.01}
	[ 5]: 0.891 (+/-0.003)

We let this code run on the gpucluster server and got the following results:

![random forest results](mnist_quest/RandomForest.png)

  
With a score of 0.96863 we feel this is a pretty good result! Nonetheless, we might be able to optimize it. The search above was done using RandomizedSearchCV. Let's try choosing a couple parameters close to our result and performing a GridSearch with them.



In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from time import time

forest = RandomForestClassifier()

tuning_parameters = {
    'n_estimators' : [1000, 1500, 2000],
    'min_samples_split' : [2, 3, 5]
}

grid_tuned = GridSearchCV(forest,
                          tuning_parameters,
                          cv=5,
                          scoring='f1_micro',
                          verbose=2,
                          n_jobs=-1)


start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
SEARCH TIME: 4508.03 sec

Best model set found on train set:

	best parameters={'min_samples_split': 2, 'n_estimators': 1500}
	best 'f1_micro' score=0.9686530612244898
	best index=1

Best estimator CTOR:
	RandomForestClassifier(n_estimators=1500)

Grid scores ('f1_micro') on development set:
	[ 0]: 0.968 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 1000}
	[ 1]: 0.969 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 1500}
	[ 2]: 0.968 (+/-0.003) for {'min_samples_split': 2, 'n_estimators': 2000}
	[ 3]: 0.968 (+/-0.003) for {'min_samples_split': 3, 'n_estimators': 1000}
	[ 4]: 0.968 (+/-0.003) for {'min_samples_split': 3, 'n_estimators': 1500}
	[ 5]: 0.968 (+/-0.003) for {'min_samples_split': 3, 'n_estimators': 2000}
	[ 6]: 0.968 (+/-0.003) for {'min_samples_split': 5, 'n_estimators': 1000}
	[ 7]: 0.968 (+/-0.002) for {'min_samples_split': 5, 'n_estimators': 1500}
	[ 8]: 0.967 (+/-0.003) for {'min_samples_split

This did not seem to improve our score by much. In our final test we will test the DecisionTreeClassifier.

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from time import time

tree = DecisionTreeClassifier()

tuning_parameters = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best'],
    'min_samples_split' : [2, 3, 5],
    'min_samples_leaf' : [1, 2, 3, 4],
    'max_features' : ['sqrt', 'log2']
}

grid_tuned = GridSearchCV(tree,
                          tuning_parameters,
                          cv=5,
                          scoring='f1_micro',
                          verbose=2,
                          n_jobs=6)


start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
SEARCH TIME: 40.16 sec

Best model set found on train set:

	best parameters={'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
	best 'f1_micro' score=0.8317346938775512
	best index=24

Best estimator CTOR:
	DecisionTreeClassifier(criterion='entropy', max_features='sqrt')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.819 (+/-0.005) for {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
	[ 1]: 0.821 (+/-0.015) for {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'best'}
	[ 2]: 0.824 (+/-0.014) for {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
	[ 3]: 0.815 (+/-0.010) for {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter'

: 

Sadly, this score is even worse than the scores we had before. We'll have to stick with our RandomTreeClassifier, with a score of 0.986.